<a href="https://colab.research.google.com/github/FranciscoFoz/Projeto_Final_Bootcamp_Data_Science_Alura/blob/main/Notebooks/Notebook_de_Funcoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color= MediumSpringGreen		size = 6> Bootcamp Data Science Alura - Projeto Final </font>

# <font color= MediumSpringGreen		size = 6> Notebook de funções

## <font color= MediumSpringGreen> 1. Funções de manipulação de dados


### <font color= MediumSpringGreen> 1.1 remover_pacientes_com_primeira_janela_positiva_para_UTI

Esta função irá remover os pacientes que possuam resultado positivo(1) para entrada na UTI dentro da primeira janela (0-2)

In [ ]:
def remover_pacientes_com_primeira_janela_positiva_para_UTI(dados):
  a_remover = dados.query("WINDOW == '0-2' and ICU == 1")['PATIENT_VISIT_IDENTIFIER'].values            #Query dos pacientes que possuam a primeira janela e resultado positivo
  dados_limpos = dados.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")                              #Query removendo os pacientes na condição acima
  return dados_limpos

### <font color= MediumSpringGreen> 1.2 preenche_tabela
Esta função irá preencher a tabela onde esteja NAN com os valores anteriores e depois com os posteriores.

In [3]:
def preenche_tabela(dados):
  features_continuas_colunas = dados.iloc[:,4:-2].columns                                                                                                   #Selecionar colunas de variáveis contínuas
  features_continuas = dados.groupby('PATIENT_VISIT_IDENTIFIER', as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')  #Preencher os dados NAN da tabela
  features_categoricas = dados.iloc[:,:4]                                                                                                                   #Selecionar as categóricas
  saida = dados.iloc[:, -2:]                                                                                                                                #Selecionar os dados de saída
  dados_finais = pd.concat([features_categoricas,features_continuas,saida],ignore_index=True,axis=1)                                                        #Concatenar os dados anteriores
  dados_finais.columns = dados.columns                                                                                                                      #Renomear as colunas
  return dados_finais

### <font color= MediumSpringGreen> 1.2 prepare_window
Esta função irá localizar todas as linhas que possuam janela entre "0-2" e UTI igual a 1, retornando as linhas.

In [8]:
def prepare_window(rows):                                                   
  if(np.any(rows['ICU'])):                                                      
    rows.loc[rows['WINDOW']=='0-2','ICU'] = 1                                   #Condição se para localizar as linhas com os filtros desejados.
  return rows.loc[rows['WINDOW'] == '0-2']

### <font color= MediumSpringGreen> 1.3 excluir_coluna_id_paciente
Esta função irá excluir a coluna do ID do paciente.

In [1]:
def excluir_coluna_id_paciente(dados):
  dados_sem_coluna_id_paciente = dados.drop(['PATIENT_VISIT_IDENTIFIER'],axis=1)      #Excluir coluna do id do paciente
  return dados_sem_coluna_id_paciente                               

### <font color= MediumSpringGreen> 1.4 transformar_AGE_PERCENTIL_em_dados_categoricos
Esta função irá excluir a coluna do ID do paciente.

In [5]:
def transformar_AGE_PERCENTIL_em_dados_categoricos(dados):
  dados.AGE_PERCENTIL = dados.AGE_PERCENTIL.astype('category').cat.codes                              #Transformar coluna AGE PECENTIL em dados categóricos
  return dados

### <font color= MediumSpringGreen> 1.5 remover_variaveis_correlacionadas
Esta função irá filtrar os dados entre as variáveis, excluindo: "AGE_ABOVE65","AGE_PERCENTIL", "GENDER", "WINDOW" e "ICU".

Criará uma matriz de correlação entre eles, permanecendo apenas com o triângulo superior.

Excluirá as colunas com correção passada como parâmetro.


In [6]:
def remover_variaveis_correlacionadas(dados,valor_de_corte):

  matriz_corr = dados.iloc[:,3:-2].corr().abs()                                                               #Filtrar dados correlacionando-os e transformando-os em números absolutos.
  matriz_superior = matriz_corr.where(np.triu(np.ones(matriz_corr.shape),k=1).astype(np.bool))                #Criar matriz superior da matriz de correlação
  excluir = [coluna for coluna in  matriz_superior if any(matriz_superior[coluna] > valor_de_corte)]          #Selecionar colunas dentro da matriz superior que contenham alta correlação

  return dados.drop(excluir,axis=1)                                                                           #Retornar o dataframe sem as colunas com alta correlação

## <font color= MediumSpringGreen> 2. Funções de criação de visualizações


### <font color= MediumSpringGreen> 2.1 grafico_total_pacientes
Esta função criará um gráfico de cartão com o total de pacientes

In [ ]:
def grafico_total_pacientes():

  plt.figure(figsize=(7, 3),                                                    #Ajustando o tamanho
            facecolor='white')                                                  #Escolhendo a cor do fundo

  for spine in plt.gca().spines.values():                                       #Removendo os eixos do gráfico
      spine.set_visible(False)

  plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False) #Excluindo os valores dos eixos


  plt.text(0, 1.03,                                                             #Ajustando posição dos eixos
          'Número total de pacientes',                                          #Colocando o título
          fontsize=25,                                                          #Formatando o tamanho  
          color='black',                                                        #Formatando a cor
          weight="bold")                                                        #Formatando em negrito

  plt.text(0, 0.25,                                                             #Ajustando posição dos eixos
          '$\\bf{353}$',                                                        #Colocando número com formatação em negrito com auxílio de regex                     
          fontsize=130,                                                         #Formatando o tamanho
          color='#4682B4')                                                      #Formatando a cor
  plt.show()

### <font color= MediumSpringGreen> 2.2 grafico_quantidade_entrada_e_nao_entrada_UTI
Esta função criará um gráfico de com a quantidade de pacientes que deram e não deram entrada na UTI.

In [7]:
def grafico_quantidade_entrada_e_nao_entrada_UTI():
                                           
  tabela_freq_entrada_UTI = pd.DataFrame(dados_limpos_sem_corr['ICU'].value_counts())     #Criar DataFrame com a quantidade
  tabela_freq_entrada_UTI.columns = ['UTI']                                               #Renomear coluna
  tabela_freq_entrada_UTI.index = ['Não','Sim']                                           #Renomear índice

  plt.figure(facecolor='white')                                                  #Escolhendo a cor do fundo
  tabela_freq_entrada_UTI.T.plot(kind='barh',                                    #Plotar o gráfico
                                      stacked=True,                              #Empilhar as barras
                                      figsize=(15,6),                            #Ajustar o tamanho
                                      color=['#808080','#800000'],               #Escolher a cor
                                      fontsize=15,                               #Ajustar o tamanho da fonte
                                      legend=None)                               #Excluir legenda

  for spine in plt.gca().spines.values():                                        #Remover os eixos do gráfico
      spine.set_visible(False)

  quantidade_sem_entrada = tabela_freq_entrada_UTI['UTI'][0]                                 
  quantidade_com_entrada = tabela_freq_entrada_UTI['UTI'][1]

  sem_entrada_percentual = str(round(quantidade_sem_entrada/tabela_freq_entrada_UTI['UTI'].sum(),2)*100)   
  com_entrada_percentual = str(round(quantidade_com_entrada/tabela_freq_entrada_UTI['UTI'].sum(),2)*100)    


                        
  plt.text(quantidade_sem_entrada/2.2,0,                                         #Escolher a posição x e y do texto de %
          sem_entrada_percentual + '%',                                          #Plotar o texto com o %
          fontweight ='bold',                                                    #Formatar para negrito
          fontsize = 20)                                                         #Aumentar a fonte

  plt.text(tabela_freq_entrada_UTI['UTI'].sum() - quantidade_sem_entrada/1.8,0,  #Escolher a posição x e y do texto
          com_entrada_percentual + '%',                                          #Plotar o texto com o %
          fontweight ='bold',                                                    #Formatar para negrito
          fontsize = 20)                                                         #Aumentar a fonte

  plt.yticks([])                                                                 #Excluir valores do eixo y
  plt.xticks([])                                                                 #Excluir valores do eixo x

  plt.text(-0.3, -0.4,                                                           #Escolher posição   
          'Do total de $\\bf{ 353 \\ pacientes}$.' ,                             #Texto com regex
          fontsize=25)                                                           #Escolher tamanho do texto                                            

  plt.text(-0.3, -0.5,                                                           #Escolher posição  
          '$\\bf{ 190 \\ não \\ tiveram \\ entrada}$.' ,                         #Texto com regex
          fontsize=25,                                                           #Escolher tamanho do texto 
          color='#808080')                                                       #Escolher cor

  plt.text(-0.3, -0.6,                                                           #Escolher posição
          'e $\\bf{ 163 \\ tiveram \\ entrada}$.' ,                              #Texto com regex
          fontsize=25,                                                           #Escolher tamanho do texto
          color='#800000')                                                       #Escolher cor
          
  plt.title('Quantidade de pacientes com entrada na UTI',  #Colocar o título
            fontsize=30,                                                         #Tamanho do texto
            loc='left',                                                          #Ajustar do lado esquerdo
            color='#808080')                                                     #Escolher a cor

  plt.show()